In [3]:
from cyvcf2 import VCF
from collections import Counter

In [4]:
## load file into notebook
infile = "../../../stripes_AIL_gg6a/F8/stripes_input/all_f8_only_founder_snps.vcf.bgz"
vcf = VCF(infile, threads=5)

In [5]:
#get sample_names
sample = vcf.samples
sampl = [i.rsplit("/",1)[1].split("_")[0] for i in sample] # adjust this if naming convention of files differs

d=dict(Counter(sampl))
#get all samples that are duplicates
dups = [key for key, item in d.items() if item>1]


In [6]:
#check if fastq files for both exist
for duplicate in dups:
    print(duplicate)
    !find /mnt/bbg/AIL_reseq/ -name *{duplicate}* #ipython, because i cant be bothered to use os right now

5215
/mnt/bbg/AIL_reseq/F0_F19_lowCov/fastq.processed/fc3/5215_F8_S41_L001_R1_001.fastq.gz
/mnt/bbg/AIL_reseq/F0_F19_lowCov/fastq.processed/fc3/5215_F8_S41_L001_R2_001.fastq.gz
/mnt/bbg/AIL_reseq/F0_F19_lowCov/fastq.processed/testplate/5215_F8_S49_L005_R1_001.fastq.gz
/mnt/bbg/AIL_reseq/F0_F19_lowCov/fastq.processed/testplate/5215_F8_S49_L005_R2_001.fastq.gz
/mnt/bbg/AIL_reseq/F0_F19_lowCov/fastq.processed/all/085215_forward.fastq.gz
/mnt/bbg/AIL_reseq/F0_F19_lowCov/fastq.processed/all/085215_reverse.fastq.gz
5311
/mnt/bbg/AIL_reseq/F0_F19_lowCov/fastq.processed/fc3/5311_F8_S82_L001_R1_001.fastq.gz
/mnt/bbg/AIL_reseq/F0_F19_lowCov/fastq.processed/fc3/5311_F8_S82_L001_R2_001.fastq.gz
/mnt/bbg/AIL_reseq/F0_F19_lowCov/fastq.processed/testplate/5311_F8_S50_L005_R1_001.fastq.gz
/mnt/bbg/AIL_reseq/F0_F19_lowCov/fastq.processed/testplate/5311_F8_S50_L005_R2_001.fastq.gz
/mnt/bbg/AIL_reseq/F0_F19_lowCov/fastq.processed/all/153110_forward.fastq.gz
/mnt/bbg/AIL_reseq/F0_F19_lowCov/fastq.processe

In [ ]:

# run vcftools to check which plate/copy was better in missingness:

! vcftools --missing-indv  --vcf {infile}

In [13]:
with open("../../../stripes_AIL_gg6a/F8/stripes_input/out.imiss", "r") as handle: 
    qual = handle.read().split("\n")
    qual.pop(0)
    qual = [i.split("\t") for i in qual]     

In [14]:
#check which samples are bad:
for i in qual:     
    if len(i)>1: 
        if i[0].rsplit("/",1)[1].split("_")[0] in dups: 
            print(i[0]) 
            print(i[4]) # fraction of called variants 


/home/tilman/storage/F8/bams/5215_F8_S49_L005_call2.bam
0.526314
/home/tilman/storage/F8/bams/5311_F8_S82_L001_call2.bam
0.726071
/home/tilman/storage/F8/bams/5167_F8_S1_L001_call2.bam
0.729484
/home/tilman/storage/F8/bams/5311_F8_S50_L005_call2.bam
0.452693
/home/tilman/storage/F8/bams/5167_F8_S48_L005_call2.bam
0.443444
/home/tilman/storage/F8/bams/5215_F8_S41_L001_call2.bam
0.833583


In [19]:
## kicking out:
exclude_samples = ["/home/tilman/storage/F8/bams/5215_F8_S49_L005_call2.bam",
"/home/tilman/storage/F8/bams/5311_F8_S50_L005_call2.bam",
"/home/tilman/storage/F8/bams/5167_F8_S48_L005_call2.bam"]

remove_samples = "../../../stripes_AIL_gg6a/F8/stripes_input/remove_F8.txt"

with open(remove_samples, "w") as handle:
    handle.write("\n".join(exclude_samples))

In [25]:
output = infile.split(".vcf")[0]+"_removed_dups.vcf"
print(output)

../../../stripes_AIL_gg6a/F8/stripes_input/all_f8_only_founder_snps_removed_dups.vcf


In [ ]:
## filtering out the bad duplicate samples using bcftools

In [29]:
! bcftools view --samples-file ^{remove_samples} -Ov {infile} -o {output}

In [30]:
vcf2 = VCF(output, threads=4)

In [31]:
print(len(vcf.samples))

379

In [34]:
print(len(vcf.samples))
print(len(vcf2.samples))  #cool, i actually filtered them out.
samples = vcf2.samples 

379
376


In [35]:
samples = vcf2.samples 
sampl2 = [i.rsplit("/",1)[1].split("_")[0]+"08" for i in samples] # adjust this if naming convention of files differs


In [37]:
Counter(sampl2).most_common(5)  ## cool, no duplicates.

[('529508', 1), ('519808', 1), ('542608', 1), ('537908', 1), ('552408', 1)]

In [7]:
new_samples = infile.rsplit("/",1)[0]+"/new_names_F8.txt"

In [8]:
new_samples

'../../../stripes_AIL_gg6a/F8/stripes_input/new_names_F8.txt'

In [44]:
with open(new_samples, "w") as handle:
    handle.write("\n".join(sampl2))

In [45]:
output2 = output.rsplit(".",1)[0]+"_new_names.vcf"

In [48]:
! bcftools reheader --samples {new_samples} -o {output2} {output}  ###rename samples

In [49]:
!grep "#CHROM" {output2} #yay

#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	529508	519808	542608	537908	552408	540708	550308	549908	549508	529908	539808	517208	550708	528408	531208	544708	535808	558008	527908	531408	561508	554008	562808	538808	536308	558208	533408	534608	545008	536408	552308	560808	556008	519608	541608	535508	541008	531708	521008	518508	560208	558108	546008	547708	530208	561008	547108	537808	554908	550208	536808	554508	532308	520808	522108	528808	544308	543208	529008	540608	520608	552808	546408	548408	538308	556408	517808	546708	531008	530608	532008	559008	535608	520908	544008	537008	555908	549208	535208	531608	540408	542908	534208	557308	542008	535408	531108	546508	559308	529408	560008	553908	543308	517708	562308	529208	530708	530108	557408	547208	542108	546308	545808	537408	558908	533908	532608	543408	553508	554308	558808	560508	544508	528308	551508	549408	559708	534808	545708	561608	534308	557108	530908	558308	520708	542708	539108	563208	519108	557608	561908	547608	545208	544808	559508	557808	51

In [50]:
##now change the vcf and the sample-file for stripes & we're set.